In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [2]:
pip install h5py

In [3]:
# Membaca dataset
df = pd.read_csv('modified_set_kerja.csv')

In [4]:
# Preprocessing teks
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = text.strip()
    return text

df['processed_description'] = df['Job Description'].apply(preprocess_text)


In [5]:
# Tokenisasi dan Padding
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 100
tokenizer = Tokenizer(num_words=MAX_WORDS)
tokenizer.fit_on_texts(df['processed_description'])
sequences = tokenizer.texts_to_sequences(df['processed_description'])
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)


In [6]:
# Label Encoding untuk target (Nama Pekerjaan)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Nama Pekerjaan'])


In [7]:
# Membagi data menjadi data latih dan uji
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [8]:
# Membangun Model
model = Sequential()
model.add(Embedding(MAX_WORDS, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(LSTM(64, dropout=0.2))
model.add(Dense(len(np.unique(labels)), activation='softmax'))  # Jumlah neuron sama dengan jumlah label unik

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Training Model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)

Epoch 1/100
3/3 [==============================] - 7s 1s/step - loss: 4.0548 - accuracy: 0.1125 - val_loss: 4.0507 - val_accuracy: 0.2000
Epoch 2/100
3/3 [==============================] - 1s 181ms/step - loss: 4.0321 - accuracy: 0.4250 - val_loss: 4.0467 - val_accuracy: 0.2500
Epoch 3/100
3/3 [==============================] - 1s 169ms/step - loss: 4.0072 - accuracy: 0.5000 - val_loss: 4.0424 - val_accuracy: 0.2500
Epoch 4/100
3/3 [==============================] - 1s 190ms/step - loss: 3.9784 - accuracy: 0.4000 - val_loss: 4.0375 - val_accuracy: 0.1500
Epoch 5/100
3/3 [==============================] - 1s 191ms/step - loss: 3.9368 - accuracy: 0.3250 - val_loss: 4.0368 - val_accuracy: 0.0500
Epoch 6/100
3/3 [==============================] - 1s 235ms/step - loss: 3.8517 - accuracy: 0.1875 - val_loss: 4.1049 - val_accuracy: 0.0000e+00
Epoch 7/100
3/3 [==============================] - 1s 198ms/step - loss: 3.7266 - accuracy: 0.0625 - val_loss: 4.2952 - val_accuracy: 0.0000e+00
Epoch 8/

In [10]:
model.save("model_1.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
